# 1.Importera de externa paketen


Kör du koden för första gången måste du köra följande tre pip install kommandon och sedan starta trycka på restart Kernel. Börjar koden plötsligt att funka kan det vara värt att köra detta igen. 

In [163]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [164]:
pip install styleframe

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [165]:
pip install xlsxwriter

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [166]:
import pandas as pd
import numpy as np
import datamodels as dm
import openpyxl as op
import datetime
import time
import math
import glob
import os 

#Interna AP4 bibliotek 
from datamodels import DimensionReader
from datamodels import NewPearlReader
from styleframe import StyleFrame

# 2.Skapa databasläsarna

In [167]:
scd = DimensionReader()
p = dm.NewPearlReader()

# 3.Definera datum

In [168]:
todaysDate = datetime.datetime.now()
#todaysDate = datetime.datetime(2022, 6, 13)
#manualCalculationDate = datetime.datetime(2022, 6, 13)

Flytta # tecknet till "todaysDate = datetime.datetime(2022, 4, 14)" och ta bort det från "todaysDate = date.today()" för att köra dagens datum.
Ta bort # tecknet om man vill hård sätta beräknat värde till ett visst datum och inte använda logiken för att ta fram senast bankdagen

# 4.Simcorp: Läs in svenska irreguljära icke bankdagar 

In [169]:
query = """
SELECT IRREGULAR_NONBANKING_DAYS FROM SCDAT.A_HOLIDAYS
WHERE CALENDAR = 'SE'
"""

In [170]:
dfLimitik = scd.read(
  query
  )

Alla svenska irreguljära icke bankdagar

In [171]:
for date in dfLimitik.irregular_nonbanking_days:
    print(date)

1998-12-24 00:00:00
1998-12-25 00:00:00
1998-12-31 00:00:00
1999-01-01 00:00:00
1999-01-06 00:00:00
1999-04-02 00:00:00
1999-04-05 00:00:00
1999-05-13 00:00:00
1999-05-24 00:00:00
1999-06-25 00:00:00
1999-12-24 00:00:00
1999-12-31 00:00:00
2000-01-06 00:00:00
2000-04-21 00:00:00
2000-04-24 00:00:00
2000-05-01 00:00:00
2000-06-01 00:00:00
2000-06-12 00:00:00
2000-06-23 00:00:00
2000-12-25 00:00:00
2000-12-26 00:00:00
2001-01-01 00:00:00
2001-04-13 00:00:00
2001-04-16 00:00:00
2001-05-01 00:00:00
2001-05-24 00:00:00
2001-06-04 00:00:00
2001-06-22 00:00:00
2001-12-24 00:00:00
2001-12-25 00:00:00
2001-12-26 00:00:00
2001-12-31 00:00:00
2002-01-01 00:00:00
2002-03-29 00:00:00
2002-04-01 00:00:00
2002-05-01 00:00:00
2002-05-09 00:00:00
2002-05-20 00:00:00
2002-06-21 00:00:00
2002-12-24 00:00:00
2002-12-25 00:00:00
2002-12-26 00:00:00
2002-12-31 00:00:00
2003-01-01 00:00:00
2003-01-06 00:00:00
2003-04-18 00:00:00
2003-04-21 00:00:00
2003-05-01 00:00:00
2003-05-29 00:00:00
2003-06-09 00:00:00


# 5.Ta fram senaste bankdagen

In [172]:
offset = max(1, (todaysDate.weekday() + 6) % 7 - 3)
timedelta = datetime.timedelta(offset)
lastWeekdayDate = todaysDate - timedelta

for index in dfLimitik.irregular_nonbanking_days: 
    if(pd.to_datetime(index).date() == lastWeekdayDate.date()):
        lastWeekdayDate = pd.to_datetime(lastWeekdayDate)-datetime.timedelta(days=1)
        if lastWeekdayDate.weekday() > 4:
            offset = max(1, (lastWeekdayDate.weekday() + 6) % 7 - 3)
            timedelta = datetime.timedelta(offset)
            lastWeekdayDate = lastWeekdayDate - timedelta
            for index in dflimitik.irregular_nonbanking_days: 
                if(pd.to_datetime(index).date() == lastWeekdayDate.date()):
                    lastWeekdayDate = pd.to_datetime(lastWeekdayDate)-datetime.timedelta(days=1)
                    for index in dflimitik.irregular_nonbanking_days: 
                        if(pd.to_datetime(index).date() == lastWeekdayDate.date()):
                            lastReportedDate = pd.to_datetime(lastWeekdayDate)-datetime.timedelta(days=1)

# 6.Konvertera datumen till olika format 

In [173]:
if 'manualCalculationDate' in locals():
    lastWeekdayDate = manualCalculationDate

todaysDateWithLine = todaysDate.strftime('%Y-%m-%d')
todaysDateNoLine = todaysDate.strftime('%Y%m%d')
lastWeekdayDateWithLine = lastWeekdayDate.strftime('%Y-%m-%d')
lastWeekdayDateNoLine = lastWeekdayDate.strftime('%Y%m%d')
lastWeekdayDateWithLineUS =  lastWeekdayDate.strftime('%d-%m-%Y')

# 7.Definera input filer

In [174]:
fileshare = '/home/jovyan/shared/Kort Netto Beräkning/'

In [175]:
#Det är den rad som bestämmer vart vi hämtar senaste Reported Value filen samt filen fil börjar vi kollar efter
lastReportedDatefileAtAll= '/home/jovyan/shared/Kort Netto Beräkning/Reported Values/ReportedPositions_' + '*'

In [176]:
#Hämtar senaste filen för en viss dag. Reported File är alltid senast skapade filen 
list_of_files = glob.glob(lastReportedDatefileAtAll) 
ReportedPositionsExcel = max(list_of_files, key=os.path.getctime)

# 8.Kontroll datum - Om något blir fel, kolla att infon nedan stämmer

In [177]:
print("Idag är: " + str(todaysDateWithLine) + ", Beräkningen är för: " + str(lastWeekdayDateWithLine))
print("Använda ReportedPositionsExcel: " + ReportedPositionsExcel)

Idag är: 2022-06-16, Beräkningen är för: 2022-06-15
Använda ReportedPositionsExcel: /home/jovyan/shared/Kort Netto Beräkning/Reported Values/ReportedPositions_20220615140759.xlsx


Om ovan av någon anlending är fel så använd variabeln manualCalculationDate några rutor upp

In [178]:
OMXS30_Static_Names = fileshare + 'StaticFiles/OMXS30_Static_Names.xlsx'
UBSFile= fileshare + 'Filetransfer/FileUBS/UBS_AMC_Valuation_QQUBCSAPAQLS_' + lastWeekdayDateWithLineUS +'.xls'
#OMXS30_Nasdaq_File_CSV = fileshare + 'Filetransfer/FileNasdaqCSV/OMXS30_EOD_'+ lastWeekdayDateNoLine + '.csv'
OMXS30_Nasdaq_File = fileshare + 'Filetransfer/FileNasdaq/EODWeightings_'+ lastWeekdayDateNoLine + '_OMXS30.xlsx'

#Avkommentera om en specefik fil behöver anges 
#OMXS30_Nasdaq_File = fileshare + 'Filetransfer/FileNasdaq/EODWeightings_20220509_OMXS30.xlsx'
#UBSFIL = fileshare + 'Filetransfer/FileUBS/UBS_AMC_Valuation_QQUBCSAPAQLS_09-05-2022.xls'
#ReportedPositionsExcel = fileshare + 'Reported Values/ReportedPositions_20220509.xlsx'

print(OMXS30_Static_Names)
print(OMXS30_Nasdaq_File)
#print(OMXS30_Nasdaq_File_CSV)
print(UBSFile)
print(ReportedPositionsExcel)

/home/jovyan/shared/Kort Netto Beräkning/StaticFiles/OMXS30_Static_Names.xlsx
/home/jovyan/shared/Kort Netto Beräkning/Filetransfer/FileNasdaq/EODWeightings_20220615_OMXS30.xlsx
/home/jovyan/shared/Kort Netto Beräkning/Filetransfer/FileUBS/UBS_AMC_Valuation_QQUBCSAPAQLS_15-06-2022.xls
/home/jovyan/shared/Kort Netto Beräkning/Reported Values/ReportedPositions_20220615140759.xlsx


Dessa filer styr hela programmet. Om man vill köra med andra än de genererade namnen så använd de som finns kommenterade ovan. ReportedPositionsExcel genererar från körningen dagen innan och är inte nödvändig för att koden ska funka, man kan fortfarande läsa ut resultatet 

#  9.Skapa klasser


In [179]:
class KortNettoBeräkningsObjekt:
    def __init__(self, symbol, closingPrice, marketValue, indexShares, indexWeight, companyName,isin,
                assetName,PartyNameInSCD,BloombergName,PostionOMXS30,TotalNumberOutstandingSharesPerIssuer,OMX30Nominellt,
                TotalOutstandsingSharesSCB = 0,AvistaInnehav = 0, TRSLongValue = 0,TRSShortWeight = '',
                 TRSShortValue = 0):
        #,OMX30Nominellt = 0

        #Från NASDAQ filen
        self.symbol = symbol 
        self.closingPrice = closingPrice
        self.marketValue = marketValue
        self.indexShares = indexShares
        self.indexWeight = indexWeight
        self.companyName = companyName
        self.isin = isin
        
        #Från Statiska filen
        self.assetName = assetName
        self.PartyNameInSCD = PartyNameInSCD
        self.BloombergName = BloombergName.split(",")
        self.BloombergNameWithEquity = [s + ' Equity' for s in self.BloombergName]
        self.OMX30Nominellt = OMX30Nominellt
        
        #Från Simcorp
        self.PostionOMXS30 = PostionOMXS30
        self.TotalOutstandsingSharesSCB = TotalOutstandsingSharesSCB
        self.AvistaInnehav = AvistaInnehav
             
        #Från UBS fil
        self.TRSLongValue = TRSLongValue
        
        #Från Pearl
        self.TRSShortWeight = TRSShortWeight     
        self.TRSShortValue = TRSShortValue                             
            
    def as_dict(self):        
        
        return {'Symbol (Nasdaq)': self.symbol,
               'Closing Price (Nasdaq)': self.closingPrice,
               'Asset Name (Nasdaq)': self.assetName,
               'ISIN (Nasdaq)': self.isin,
               'Ticker manuellt justerad': listToString(self.BloombergName),
               'Totalt Emitterat utestående antal aktier':self.TotalOutstandsingSharesSCB,
               'Vikt(%)': self.indexWeight,
               "Postion OMXS30":self.PostionOMXS30,
               "Position i bolag via OMX30":self.getPostionInCompany(),
               "Position i bolag via OMX30 Nominellt":self.OMX30Nominellt,
               "Avistainnehav Nominellt": self.AvistaInnehav,
               "Antal aktier - kort nettoposition": self.OMX30Nominellt+self.AvistaInnehav,

                #TRS
               "TRS Långa Värde Nominellt": self.TRSLongValue,
               "TRS Kort Vikt": self.getTRSShortWeight(),
               "TRS Korta Nominellt": self.TRSShortValue,
               "TRS Korta + Långa": self.TRSLongValue+self.TRSShortValue,
               "Totalt netto OMX, avista & TRS":
                
                   self.OMX30Nominellt+
                   self.AvistaInnehav+
                   self.TRSLongValue+
                   self.TRSShortValue
               ,
               "position att rapportera":round(self.getPositiontoReport(),4), 
               "position att rapportera i %":round(self.getPositiontoReport()*100,4)  
               }
    
    def UpdateTotalOutstandsingSharesSCB(self, NewTotalOutstandsingSharesSCB):
        self.TotalOutstandsingSharesSCB = int(self.TotalOutstandsingSharesSCB)+NewTotalOutstandsingSharesSCB

    def UpdateAvistaInnehav(self, NewAvistaInnehav):
        self.AvistaInnehav = NewAvistaInnehav
        
    def UpdateTRSLongValue(self,NewTRSLongValue):
        self.TRSLongValue = self.TRSLongValue+NewTRSLongValue
        
    def UpdateTRSShortWeight(self, NewTRSShortWeight):
        self.TRSShortWeight = self.TRSShortWeight + ', ' + str(NewTRSShortWeight)     
        
    def UpdateTRSShortValue(self,NewTRSShortValue):
        self.TRSShortValue = self.TRSShortValue + NewTRSShortValue
    
    def getTRSShortWeight(self):
        if self.TRSShortWeight == '':
            return '0'
        else:
            return self.TRSShortWeight[2:]
    
    def getPostionInCompany(self):
        return (self.indexWeight*self.PostionOMXS30)/100 
        
    def getPositiontoReport(self):
        return ((self.OMX30Nominellt+self.AvistaInnehav+self.TRSLongValue+self.TRSShortValue)
                 /self.TotalOutstandsingSharesSCB)    

Om man någonsin vill ändra någon namn eller göra någon justering på beräkningen är det här den bör ske

In [180]:
#SimCorp klass

class SimCorpSharesOutstanding:
    def __init__(self, SECSHORT, SECNO, SECSERNO, ISIN, ISSVOL, PAR, DATE, BBNAME):

        self.SECSHORT = SECSHORT
        self.SECNO = SECNO
        self.SECSERNO = SECSERNO
        self.ISIN = ISIN
        self.ISSVOL = ISSVOL
        self.PAR = PAR
        self.DATE = DATE
        self.BBNAME = BBNAME

    def as_dict(self):
        return {'Security ID': self.SECSHORT,
                'Security NO.': self.SECNO,
                'Ser. NO': self.SECSERNO,
                'ISIN': self.ISIN,
                'Issued volume': self.ISSVOL,
                'Party': self.PAR,
                'Date': self.DATE,
                'BBName': self.BBNAME
                }

In [181]:
# Hjälp Klass

def listToString(theList):
    # initialize an empty string
    string = ""

    # traverse in the string
    for element in theList:
        string += element + ','

    # return string
    return string


# 10.Läs in Static filen


Vid förändringar av innehav i OMX - alt vid namnändringar - måste denna fil uppdateras

In [182]:
OMXS30StaticNames_DataFrame =  pd.read_excel(OMXS30_Static_Names)
print(OMXS30StaticNames_DataFrame)

   Asset Name Nasdaq                       Asset Name  Party name in SCD  \
0              SKF B                            AB SKF               SKF   
1                ABB                           ABB LTD           ABB LTD   
2               ALFA                     ALFA LAVAL AB        ALFA LAVAL   
3             ASSA B                     ASSA ABLOY AB        ASSA ABLOY   
4                AZN                   ASTRAZENECA PLC       ASTRAZENECA   
5             ATCO B                    ATLAS COPCO AB       ATLAS COPCO   
6             ATCO A                    ATLAS COPCO AB       ATLAS COPCO   
7           ALIV SDB                       AUTOLIV INC           AUTOLIV   
8                BOL                        BOLIDEN AB           BOLIDEN   
9             ELUX B                     ELECTROLUX AB        ELECTROLUX   
10          ESSITY B                         ESSITY AB            ESSITY   
11               EVO         EVOLUTION GAMING GROUP AB  EVOLUTION GAMING   
12          

# 11.Läs in Simcorp Limit "FI-B MAX OMX INDEX" för att hämta Postion för OMXS30

In [183]:
# Hämar Limitens unika ID
query = """
select LIMITIK from SCDAT.LIMITRULES SCDAT WHERE LIMIT ='FI-B MAX OMX INDEX'
"""

In [184]:
dflimitik = scd.read(
  query
  )

In [185]:
limitik = int(dflimitik.limitik[0])

In [186]:
queryPostionOMXS30 = """select LIMITACT from SCDAT.LIMUTILISATION SCDAT
where LIMITIK = :limitik
and LIMITDATE = TO_DATE(:usedate,'YYYY-MM-DD')
order by LIMITDATE desc"""

In [187]:
dfPostionOMXS30 = scd.read(
  queryPostionOMXS30, params={'usedate': lastWeekdayDateNoLine,'limitik':limitik}
  )
print(dfPostionOMXS30)

       limitact
0 -1.093522e+10


In [188]:
PostionOMXS30 = int(dfPostionOMXS30.limitact[0])
print('PostionOMXS30: ' + str(PostionOMXS30))

PostionOMXS30: -10935215750


# 12.Läs in Dagliga Nasdaq filen

In [189]:
wbNasdaq = op.load_workbook(OMXS30_Nasdaq_File)
wsNasdaq=wbNasdaq['Weightings'] #flikens namn

In [190]:
#skapar tom lista
OMX30List = []

# 12.1 Skapa objekt

Fastnar koden här kan det bero på att det finns ett instrument i Nasdaq filen som inte finns i den statiska mapping filen

In [191]:
for row in wsNasdaq.iter_rows(min_row=6, max_row=wsNasdaq.max_row-1):

    assetName = OMXS30StaticNames_DataFrame['Asset Name '].where(OMXS30StaticNames_DataFrame['Asset Name Nasdaq'] == row[0].value).dropna().values[0]
    PartyNameInSCD = OMXS30StaticNames_DataFrame['Party name in SCD'].where(OMXS30StaticNames_DataFrame['Asset Name Nasdaq'] == row[0].value).dropna().values[0]
    BloombergNamn = OMXS30StaticNames_DataFrame['Bloomberg namn'].where(OMXS30StaticNames_DataFrame['Asset Name Nasdaq'] == row[0].value).dropna().values[0]
        
    fixedSimCorpBBName = BloombergNamn.replace(' SW ',' SE ')
        
    TotalNumberOutstandingSharesPerIssuer= ""
    print('assetName: ' + assetName)
    print('PartyNameInSCD: ' + PartyNameInSCD)
    print('BloombergNamn: ' + fixedSimCorpBBName)
          
    symbol = row[0].value
    closingPrice = row[1].value
    marketValue = row[2].value
    indexShares = row[3].value
    companyName = row[5].value
    isin = row[16].value

    indexWeight = row[4].value              
    OMX30Nominellt =((indexWeight*PostionOMXS30/100)/closingPrice)
    
    for värde in OMX30List:
            if(värde.assetName == assetName):
                indexWeight = row[4].value +värde.indexWeight
                print("Värde med samma assetname = " + assetName + " .För över värden till det senaste värdet")
                print("Nuvarande vikt: " + str(värde.indexWeight))
                print("Ny vikt: " + str(indexWeight))
                print("Nuvarande nominellt värde: " + str(värde.OMX30Nominellt))
                print("Nytt nominellt värde: " + str(OMX30Nominellt))
                print("Row with name " + str(värde.symbol) + " will be removed")
                indexWeight = row[4].value +värde.indexWeight
                OMX30Nominellt = OMX30Nominellt + värde.OMX30Nominellt 
                symbol = värde.symbol + "," + symbol
                isin = värde.isin + "," + isin
                closingPrice = str(värde.closingPrice) + " , " + str(closingPrice)
                OMX30List.remove(värde)

    tempStorageOMX30Value = KortNettoBeräkningsObjekt(
        symbol,
        closingPrice,
        marketValue,
        indexShares,
        indexWeight,
        companyName,
        isin,
        assetName,
        PartyNameInSCD,
        fixedSimCorpBBName,
        PostionOMXS30,
        TotalNumberOutstandingSharesPerIssuer,
        OMX30Nominellt
    )
    OMX30List.append(tempStorageOMX30Value)


assetName: ABB LTD
PartyNameInSCD: ABB LTD
BloombergNamn: ABBN SE
2.968634062
-10935215750
287
assetName: ALFA LAVAL AB
PartyNameInSCD: ALFA LAVAL
BloombergNamn: ALFA SS
2.220599781
-10935215750
251.7
assetName: ASSA ABLOY AB
PartyNameInSCD: ASSA ABLOY
BloombergNamn: ASSAB SS
4.992936585
-10935215750
225
assetName: ASTRAZENECA PLC
PartyNameInSCD: ASTRAZENECA
BloombergNamn: AZN SS,AZN LN
4.287289501
-10935215750
1234
assetName: ATLAS COPCO AB
PartyNameInSCD: ATLAS COPCO
BloombergNamn: ATCOA SS,ATCOB SS
7.097285949
-10935215750
100.5
assetName: ATLAS COPCO AB
PartyNameInSCD: ATLAS COPCO
BloombergNamn: ATCOA SS,ATCOB SS
2.871297712
-10935215750
87.46
Värde med samma assetname = ATLAS COPCO AB .För över värden till det senaste värdet
Nuvarande vikt: 7.097285949
Ny vikt: 9.968583661
Nuvarande nominellt värde: -7722423.193209801
Nytt nominellt värde: -3590013.7163504884
Row with name ATCO A will be removed
assetName: AUTOLIV INC
PartyNameInSCD: AUTOLIV
BloombergNamn: ALIV SS
0.78239594
-1093

# 13.Hämtar utestående aktier 

In [192]:
#Konverar samtliga instrument i 'Party name in SCD' för att kunna använda dom i SQL frågan 
joined_string = "'"+ "','".join(OMXS30StaticNames_DataFrame['Party name in SCD']) + "'"

In [193]:
queryPARIK = """
select PARIK FROM scdat.PARTNERS WHERE par In (%s)
""" % joined_string

In [194]:
# Hämtar ID för Issuer i DB
dfPARIK = scd.read(
  queryPARIK,params={'joined_string': joined_string}
  )

In [134]:
#Konverar resultatet från frågan ovan för att kunna använda dom i nästa SQL frågan 
converted_list = [str(element) for element in dfPARIK['parik']]
joined_string_parik = "'"+ "','".join(converted_list) + "'"

In [196]:
#Hämtar utestående aktier från databasen

query = """SELECT A.SECSHORT, A.SECNO, A.SECSERNO, A.ISIN, C.SHAREISSVOL, B.PAR, C.SHAREDATE, A.BLOOMBERGCODE, A.SECNAME
FROM SCDAT.SECURITIES A
join SCDAT.PARTNERS B on a.PARIK = B.PARIK
JOIN SCDAT.SHARECAPITAL C ON A.SECIK = C.SECIK
WHERE A.SECTYPEIK in (SELECT SECTYPEIK FROM SCDAT.SECTYPES WHERE SECTYPE = 'EQUITY')
and A.hidden = 0 
AND A.PARIK in (%s)
AND SHAREDATE =TO_DATE(:usedate,'YYYY-MM-DD')
ORDER BY A.SECNO""" % (joined_string_parik)

In [197]:
readLimitik = scd.read(
    query,params={'limitik':limitik,'usedate': lastWeekdayDateNoLine}
  )

In [229]:
SimcorpTempStorage = 0
SimcorpList = []

for i,row in readLimitik.iterrows(): 
    #print(row[0])
    #print(row[1])
    #print(row[2])
    #print(row[3])
    #print(row[4])
    #print(row[5])
    #print(row[6]) 
    #print(row[7])
    tempStorageSimCorpSharesOutstanding = SimCorpSharesOutstanding(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7])
    SimcorpList.append(tempStorageSimCorpSharesOutstanding)

In [199]:
for index in OMX30List:
        for SimcorpTempStorage in SimcorpList:
            if(index.PartyNameInSCD == SimcorpTempStorage.PAR):
                #OM VI VILL ÄNDRA BASEN PÅ NÅGON AV ISSUERS NEDAN UPPDATERAR VI BELOPPEN NEDAN, EXEMPELVIS VID 
                #FALL AV INLÖSENAKTIER, INCITAMENTSAKTIER OSVOSV. se till att göra korrekta "indents" också
                # använd namnformat PartyName in SCD, column C i Static Data filen
                #if(index.PartyNameInSCD == "ELECTROLUX"):
                 #   index.UpdateTotalOutstandsingSharesSCB(300000000)
                  #  print(index.PartyNameInSCD + ' Add ' + str(300000000))
                #else:
                index.UpdateTotalOutstandsingSharesSCB(SimcorpTempStorage.ISSVOL) 
                print(index.PartyNameInSCD + ' Add ' + str(SimcorpTempStorage.ISSVOL))

ABB LTD Add 1493119871.0
ABB LTD Add 471625204.0
ALFA LAVAL Add 413876823.0
ASSA ABLOY Add 1055050365.0
ASSA ABLOY Add 57525969.0
ASTRAZENECA Add 1549109281.0
ASTRAZENECA Add 361331.0
ATLAS COPCO Add 3357576384.0
ATLAS COPCO Add 1560876032.0
AUTOLIV Add 55290758.0
AUTOLIV Add 46984544.0
BOLIDEN Add 273511169.0
ELECTROLUX Add 274885045.0
ELECTROLUX Add 8192348.0
ERICSSON Add 261755983.0
ERICSSON Add 3072395752.0
ESSITY Add 641053575.0
ESSITY Add 61288914.0
EVOLUTION GAMING Add 215111115.0
GETINGE IND Add 254152373.0
GETINGE IND Add 18217200.0
H&M Add 1460672000.0
H&M Add 194400000.0
HEXAGON Add 2595227888.0
HEXAGON Add 110250000.0
INVESTOR Add 1821936744.0
INVESTOR Add 1246763376.0
KINNEVIK Add 33755432.0
KINNEVIK Add 242418116.0
KINNEVIK Add 2788199.0
NORDEA BANK Add 3789514793.0
SANDVIK Add 1254385923.0
SINCH AB Add 837380778.0
SEB Add 2154569426.0
SEB Add 24152508.0
SKANSKA Add 19660647.0
SKANSKA Add 400242425.0
SKF Add 29403933.0
SKF Add 425947135.0
SCA Add 64581994.0
SCA Add 637760

# 14.Hämta AvistaInnehav

In [200]:
query ="""SELECT L.LIMGROUPVAL, L.LIMITACTGRPVAL, U.LIMIT_REPORT_DATE 
FROM SCDAT.LIMUTILCOMPLDETAIL L, SCDAT.A_LIMITS N, SCDAT.A_LIMITS_UTILISATION U
where
N.LIMIT= 'OMX30 - BALANCE NOM'
and N.LIMIT_REF=U.LIMIT_REF
and U.LIMIT_REPORT_DATE =TO_DATE(:usedate,'YYYY-MM-DD')
and U.LIMIT_UTILISATION_REF=L.LIMUTILIK"""

In [201]:
AvistaRead = scd.read(
    query,params={'usedate': lastWeekdayDateNoLine}
  )

In [135]:
#for value in AvistaRead.iterrows():
    #print(value)

In [203]:
addedItems = []

for index in OMX30List:
    for value in AvistaRead.iterrows():
        if(index.PartyNameInSCD == value[1].limgroupval):# and index.PartyNameInSCD not in addedItems):
            print(value[1].limgroupval)
            print(value[1].limitactgrpval)
            addedItems.append(index.PartyNameInSCD)
            index.UpdateAvistaInnehav(value[1].limitactgrpval)

ABB LTD
334391.0
ALFA LAVAL
1344932.0
ASSA ABLOY
5222338.0
ASTRAZENECA
1591885.0
ATLAS COPCO
41252968.0
AUTOLIV
739491.0
BOLIDEN
107733.0
ELECTROLUX
780.0
ERICSSON
186649.0
ESSITY
37478.0
GETINGE IND
20385950.0
H&M
22363258.0
HEXAGON
3482213.0
INVESTOR
3682.0
NORDEA BANK
150000.0
SANDVIK
12048693.0
SINCH AB
32506590.0
SEB
20867329.0
SKANSKA
2557076.0
SCA
310851.0
SHB
24960.0
SWEDBANK
9146.0
TELE2
71723.0
TELIASONERA
222292.0
VOLVO
14519825.0


# 15.Hämta TRS Långa

In [204]:
UBS_DataFrame =  pd.read_excel(UBSFile, sheet_name = 'Equity',  skiprows=21)

In [205]:
for index in OMX30List:
    for namn in index.BloombergName:
        if(namn == 'ATCOA SS'):
            namn = 'ATCO A SS'
        if(namn == 'ATCOB SS'):
            namn = 'ATCO B SS'
        #print(namn)
        for row in UBS_DataFrame.iterrows():
            #print(row[1].BBG)
            if namn == row[1].BBG:
                index.UpdateTRSLongValue(row[1].Shares)
                print('Updated: ' + namn)
                print(row[1].Shares)


Updated: ASSAB SS
63405.54
Updated: ATCO A SS
235562.355
Updated: EVO SS
10011.42
Updated: HMB SS
77146.695
Updated: HEXAB SS
26500.77
Updated: INVEB SS
66742.65


# 16.GDDUWI and USD/SEK


In [206]:
UBS_DataFrame =  pd.read_excel(UBSFile, sheet_name = 'Index', skiprows=21)
usdGDDUWI = UBS_DataFrame['Total Val. (USD)'].iloc[0]
print('USD GDDUWI ' + str(usdGDDUWI))

USD GDDUWI -420153165.49752


In [207]:
queryfxrateintqp = """select  from_ccy_id,
        to_ccy_id,
        vdate,
        value
from    out_exrate
where   vdate = TO_DATE(:usedate,'YYYY/MM/DD')
    and fxset_id = 'D'
    and from_ccy_id not like '%0'
    and from_CCY_ID = 'USD'
"""

In [208]:
dffxrateintqp = p.read(
  queryfxrateintqp,params={'usedate': lastWeekdayDateNoLine}
  )
print(dffxrateintqp)

  from_ccy_id to_ccy_id      vdate    value
0         USD       SEK 2022-06-15  10.2339


In [209]:
exchagnerate = dffxrateintqp.value[0]
print('Exchagnerate ' + str(exchagnerate))

Exchagnerate 10.2339


In [210]:
sekGDDUWI = usdGDDUWI*exchagnerate
print('SED GDDUWI ' + str(sekGDDUWI))

SED GDDUWI -4299805480.38507


# 17.Hämta TRS från inläst Bloomberg fil

In [211]:
query = '''select
           calculation_date,
            nvl(isin, 'N/A') isinvalue,
            security_name,  
            nr_of_shares_today_index*foreign_inclusion_fctr*REPLACE(reserved_5, '.', ',') test
             ,
            NR_OF_SHARES_ND_INDEX * foreign_inclusion_fctr * REPLACE(reserved_5, '.', ',') * price * fx.value mv_base
            ,
            NR_OF_SHARES_ND_INDEX * foreign_inclusion_fctr * REPLACE(reserved_5, '.', ',') * price * fx.value
            / sum(NR_OF_SHARES_ND_INDEX * foreign_inclusion_fctr * REPLACE(reserved_5, '.', ',') * price * fx.value) over() weight,
            e.*


from        EXT_BM_MSCI_CONST e
inner join  out_exrate fx
    on      fx.vdate = e.calculation_date
    and     e.ISO_CCY_SYMBOL_OF_PRICE = fx.from_ccy_id   
where       calculation_date = TO_DATE(:usedate,'YYYY-MM-DD')
    and     (developed_markets_flag = 1)
    and     (STANDARD_INF_FLAG = 1 or STD_INF_FLAG_ND = 1)

'''

In [212]:
pearl_data = p.read(query,params={'usedate': lastWeekdayDateNoLine})
print(pearl_data)

     calculation_date     isinvalue           security_name          test  \
0          2022-06-15  AU000000QAN2          QANTAS AIRWAYS  9.241619e+08   
1          2022-06-15  AU000000EVN4        EVOLUTION MINING  1.833008e+09   
2          2022-06-15  AU000000FMG4  FORTESCUE METALS GROUP  1.693431e+09   
3          2022-06-15  AU000000BSL0         BLUESCOPE STEEL  4.902115e+08   
4          2022-06-15  AU000000MGR9            MIRVAC GROUP  3.942668e+09   
...               ...           ...                     ...           ...   
1510       2022-06-15  US5747951003                  MASIMO  5.259646e+07   
1511       2022-06-15  US58733R1023            MERCADOLIBRE  4.537708e+07   
1512       2022-06-15  US90384S3031             ULTA BEAUTY  5.412017e+07   
1513       2022-06-15  US9285634021                VMWARE A  2.094040e+08   
1514       2022-06-15  US1890541097               CLOROX CO  1.230579e+08   

           mv_base    weight index_family calculation_date  \
0     3.09083

In [213]:
for index in OMX30List:
    for name in index.BloombergNameWithEquity:   
        print(name)

ABBN SE Equity
ALFA SS Equity
ASSAB SS Equity
AZN SS Equity
AZN LN Equity
ATCOA SS Equity
ATCOB SS Equity
ALIV SS Equity
BOL SS Equity
ELUXB SS Equity
ELUXA SS Equity
ERICB SS Equity
ERICA SS Equity
ESSITYB SS Equity
ESSITYA SS Equity
EVO SS Equity
GETIB SS Equity
HMB SS Equity
HEXAB SS Equity
INVEB SS Equity
INVEA SS Equity
KINVB SS Equity
KINVA SS Equity
NDA SS Equity
SAND SS Equity
SINCH SS Equity
SEBA SS Equity
SKAB SS Equity
SKAA SS Equity
SKFB SS Equity
SKFA SS Equity
SCAB SS Equity
SCAA SS Equity
SHBA SS Equity
SHBB SS Equity
SWEDA SS Equity
SWMA SS Equity
TEL2B SS Equity
TEL2A SS Equity
TELIA SS Equity
VOLVB SS Equity
VOLVA SS Equity


In [214]:
queryfxrateintqpVarible = """select  from_ccy_id,
        to_ccy_id,
        vdate,
        value
from    out_exrate
where   
     fxset_id = 'D'
    and from_ccy_id not like '%0'
    and from_CCY_ID = :currency
    AND vdate = TO_DATE(:usedate,'YYYY/MM/DD')
"""

In [215]:
for index in OMX30List:
    for name in index.BloombergNameWithEquity:   
            for value in pearl_data.iterrows():
                #print(value[1].bloomberg_ticker)
                if(value[1].bloomberg_ticker == name):
                   
                    roundWeight = round(value[1].weight,6)
                    index.UpdateTRSShortWeight(roundWeight)
                    AndelExponering = sekGDDUWI*value[1].weight
                    
                    #Hämta valutakurs
                    valutaPearl = value[1].iso_ccy_symbol_of_price      
                    dffxrateintqpCurrency = p.read(queryfxrateintqpVarible,params={'usedate': lastWeekdayDateNoLine,'currency':valutaPearl})
                    kursPearl = dffxrateintqpCurrency.value[0]
                    
                    price = value[1].price
                    valueNominell = AndelExponering/(price*kursPearl)
                    index.UpdateTRSShortValue(valueNominell)
                    
                    print(name)
                    print('Värde nominell ' + str(valueNominell))
                    print('Weight: ' + str(roundWeight))
                    print('AndelExponering: ' + str(AndelExponering))
                    print('Valutan är: ' + str(valutaPearl))
                    print('Andel exponering dividerar med valutakursen ' + str(kursPearl))
                           

ABBN SE Equity
Värde nominell -13993.096126223758
Weight: 0.000932
AndelExponering: -4005904.890846213
Valutan är: CHF
Andel exponering dividerar med valutakursen 10.224187022328788
ALFA SS Equity
Värde nominell -2501.4296626939886
Weight: 0.000146
AndelExponering: -629609.8461000769
Valutan är: SEK
Andel exponering dividerar med valutakursen 1.0
ASSAB SS Equity
Värde nominell -8538.868309341833
Weight: 0.000447
AndelExponering: -1921245.3696019123
Valutan är: SEK
Andel exponering dividerar med valutakursen 1.0
AZN LN Equity
Värde nominell -13199.963098767439
Weight: 0.003737
AndelExponering: -16066226.808120446
Valutan är: GBP
Andel exponering dividerar med valutakursen 12.34924713
ATCOA SS Equity
Värde nominell -22883.337040437844
Weight: 0.000535
AndelExponering: -2299775.3725640033
Valutan är: SEK
Andel exponering dividerar med valutakursen 1.0
ATCOB SS Equity
Värde nominell -13297.559131422955
Weight: 0.00027
AndelExponering: -1163004.5216342516
Valutan är: SEK
Andel exponering di

OBS: Koden ovan kan ta någon minut att köra igen 

# 18.Skapa output-filnamn


In [216]:
currentTime = datetime.datetime.now()
hours = 2
hours_added = datetime.timedelta(hours = hours)
real_date_and_time = currentTime + hours_added
currentTime = real_date_and_time.strftime('%H%M%S')

filename = fileshare + 'Calculations/Per ' + lastWeekdayDateNoLine +  ' (kontroll)' + todaysDateNoLine + ') ' + str(currentTime)+ '.xlsx'
print('Filnamnet')
print(filename)

reportedPostionFilename = fileshare + 'Reported Values/ReportedPositions_' + lastWeekdayDateNoLine + str(currentTime) + '.xlsx'
print('reportedPostionFilename')
print(reportedPostionFilename)


Filnamnet
/home/jovyan/shared/Kort Netto Beräkning/Calculations/Per 20220615 (kontroll)20220616) 142036.xlsx
reportedPostionFilename
/home/jovyan/shared/Kort Netto Beräkning/Reported Values/ReportedPositions_20220615142036.xlsx


# 19.Skapa Sheets

In [217]:
writer = pd.ExcelWriter(filename)

outputDateFrame = pd.DataFrame([x.as_dict() for x in OMX30List])
outputDateFrame.style.to_excel(writer, sheet_name='Kort Netto Beräknings Objekt', index=False)

workbook = writer.book
worksheet  = writer.sheets['Kort Netto Beräknings Objekt']

#Definerar numberformattet 
a_format = workbook.add_format({'num_format': '#,##0.00'})
a_format.set_align('center')

#Anpassar bredden på kolumen 
for column in outputDateFrame:
    
    column_length = max(outputDateFrame[column].astype(str).map(len).max(), len(column)) 
    col_idx = outputDateFrame.columns.get_loc(column)
    writer.sheets['Kort Netto Beräknings Objekt'].set_column(col_idx, col_idx, column_length,a_format)      

In [218]:
simdimDateFrame = pd.DataFrame(x.as_dict() for x in SimcorpList)
simdimDateFrame.to_excel(writer, sheet_name='No.Shares Outstanding SCD', index=False)

#Anpassar bredden på kolumen 
for column in simdimDateFrame:
    column_length = max(simdimDateFrame[column].astype(str).map(len).max(), len(column))
    col_idx = simdimDateFrame.columns.get_loc(column)
    writer.sheets['No.Shares Outstanding SCD'].set_column(col_idx, col_idx, column_length)

In [219]:
caluclationDataframe = pd.DataFrame()

caluclationDataframe.at[""] = ""
#Lägger till en tom ruta för att få värderna att komma in
caluclationDataframe["GDDUWI i USD"] = usdGDDUWI
caluclationDataframe["Valutakurs"] = exchagnerate
caluclationDataframe["GDDUWI i SEK"] = sekGDDUWI


caluclationDataframe.to_excel(writer, sheet_name='Calculations Data', index=False)

#Anpassar bredden på kolumen 
for column in caluclationDataframe:
    column_length = max(caluclationDataframe[column].astype(str).map(len).max(), len(column))
    col_idx = caluclationDataframe.columns.get_loc(column)
    writer.sheets['Calculations Data'].set_column(col_idx, col_idx, column_length)

print(caluclationDataframe)

  GDDUWI i USD  Valutakurs  GDDUWI i SEK
 -4.201532e+08     10.2339 -4.299805e+09


In [220]:
pearl_data.to_excel(writer, sheet_name='Pearl Data', index=False)

# 20.Skapa den historiska Excelfilen

In [221]:
writer.save()

# 21.Analysera rapportbehovet

In [222]:
#Denna kod ligger här mest om man vill lägga in några kontroller

for index in OMX30List:
    _postiontoReport = index.getPositiontoReport()*100

    if(_postiontoReport< -0.1):
        print("Beräknad kort nettopositon " + index.symbol + " på nivå " + str(round(_postiontoReport,2)))

    if(index.TotalOutstandsingSharesSCB == 0):
        print("TotalOutstandsingSharesSCB för " + index.symbol + " är 0, kan vara värt att undersöka")
        

Beräknad kort nettopositon BOL på nivå -0.19
Beräknad kort nettopositon ELUX B på nivå -0.24
Beräknad kort nettopositon ERIC B på nivå -0.21
Beräknad kort nettopositon ESSITY B på nivå -0.21
Beräknad kort nettopositon EVO på nivå -0.23
Beräknad kort nettopositon INVE B på nivå -0.13
Beräknad kort nettopositon KINV B på nivå -0.2
Beräknad kort nettopositon SKF B på nivå -0.22
Beräknad kort nettopositon SCA B på nivå -0.17
Beräknad kort nettopositon SHB A på nivå -0.23
Beräknad kort nettopositon SWED A på nivå -0.23
Beräknad kort nettopositon SWMA på nivå -0.24
Beräknad kort nettopositon TEL2 B på nivå -0.21
Beräknad kort nettopositon TELIA på nivå -0.23


In [223]:
class reportValuesObjects:
    def __init__(self, name,reportedDate,previousValue,lowerValue,higherValue,calculatedValue,calculatedDalue):

        self.name = name
        self.previousValue = previousValue
        self.previousValue = previousValue
        self.lowerValue = lowerValue
        self.higherValue = higherValue,
        self.calculatedValue = calculatedValue,
        #self.calculatedValueRound = calculatedValueRound
        self.calculatedDalue = calculatedDalue
        
    def hasReportedValue(self):
        return (self.previousValue is not None)


In [224]:
ReportedPositions_DataFrame = pd.read_excel(ReportedPositionsExcel)

levelToReportNewValueAt = -0.1

#Sorterar bort oanvänd värden 
ReportedPositions_DataFrame['Reported Date'] = pd.to_datetime(ReportedPositions_DataFrame['Reported Date'], dayfirst=True)
previousvalues = ReportedPositions_DataFrame.sort_values('Reported Date').groupby('Asset Name (Nasdaq)').last()
previousvalues = previousvalues.loc[(previousvalues['Reported percent'] < 0.00)]

print("Filen nedan är den som används i denna körning")
print(ReportedPositionsExcel)
print('Senaste rapporterade värde')
print(previousvalues)

Filen nedan är den som används i denna körning
/home/jovyan/shared/Kort Netto Beräkning/Reported Values/ReportedPositions_20220615140759.xlsx
Senaste rapporterade värde
                     Reported percent Reported Date
Asset Name (Nasdaq)                                
BOL                              -0.1    2022-06-07
ELUX B                           -0.2    2022-05-17
ERIC B                           -0.2    2022-04-04
ESSITY B                         -0.2    2022-05-04
EVO                              -0.2    2022-05-05
INVE B                           -0.1    2022-05-03
KINV B                           -0.2    2022-06-12
SCA B                            -0.1    2022-05-04
SHB A                            -0.2    2022-05-05
SKF B                            -0.2    2022-05-04
SWED A                           -0.2    2022-05-05
SWMA                             -0.2    2022-05-06
TEL2 B                           -0.2    2022-05-03
TELIA                            -0.2    2022-05-04

# 22. Behöver något rapporteras idag? 

In [225]:
reportedValues = []
anythingNewToReport = False

for index in OMX30List:

        name = index.symbol
        reportedDate = None
        previousValue = None
        lowerValue = None
        higherValue = None
        #Den bort kommenterade raden nedan kan användas för att testa koden
        postiontoReport = index.getPositiontoReport()*100#-0.3
        
        for row in previousvalues.iterrows():
            if(index.symbol == row[0]):
                reportedDate = row[1]['Reported Date']
                previousValue= row[1]['Reported percent']
                lowerValue = row[1]['Reported percent']
                higherValue = row[1]['Reported percent']-0.1
        
        tempReportedvalueObjet = reportValuesObjects(
            name,
            reportedDate,
            previousValue,
            lowerValue,
            higherValue,
            postiontoReport,
            #calculatedValueRound,
            todaysDateWithLine)               
        reportedValues.append(tempReportedvalueObjet)

for row in reportedValues:
    
    calculatedValueRound = math.ceil(row.calculatedValue[0]*10)/10
    
    if(row.hasReportedValue()):
        #Ta bort dessa kommentarer om ni vill se värderna koden använder för att kolla om det är dags att rapportera
        #print("name " + row.name)
        #print("previousValue " + str(row.previousValue))
        #print("lowerValue " + str(row.lowerValue))
        #print("higherValue " + str(row.higherValue[0]))
        #print("calculatedValue " + str(row.calculatedValue[0]))

        if not (row.lowerValue > row.calculatedValue[0] > row.higherValue[0]): 
            print("Värdet har ändrats "
                          + row.name
                          + '. Värdet är ' 
                          + str(row.calculatedValue[0])
                          + ' som avrundas till '
                          + str(calculatedValueRound)
                          + ' och senaste rapporterade var ' 
                          + str(row.previousValue))
            
            new_row = {'Asset Name (Nasdaq)': row.name,
                       'Reported percent': calculatedValueRound,
                       'Reported Date': lastWeekdayDateWithLine }
            ReportedPositions_DataFrame = ReportedPositions_DataFrame.append(new_row, ignore_index=True)
            anythingNewToReport = True
            
    elif(row.calculatedValue[0] < levelToReportNewValueAt):
        print("Nytt värde att rapportera för " 
              + row.name 
              + " är " 
              + str(row.calculatedValue[0])
              + '/ '
              + str(calculatedValueRound)          
             )
        new_row = {'Asset Name (Nasdaq)': row.name,
                   'Reported percent': calculatedValueRound,
                   'Reported Date': lastWeekdayDateWithLine }
        ReportedPositions_DataFrame = ReportedPositions_DataFrame.append(new_row, ignore_index=True)
        anythingNewToReport = True

if(anythingNewToReport == False):
    print("Nothing New to report today")


Nothing New to report today


# 23.Skapa historisk rapporterad position rapport

In [136]:
#ReportedPositions_DataFrame.sort_values(by = ['Asset Name (Nasdaq)'])


In [227]:
ReportedPositions_DataFrame.sort_values(by = ['Asset Name (Nasdaq)','Reported Date'])

writerReportedPositions_DataFrame = pd.ExcelWriter(reportedPostionFilename)
ReportedPositions_DataFrame.to_excel(writerReportedPositions_DataFrame, sheet_name='Arkiv', index=False)

for column in ReportedPositions_DataFrame:
    column_length = max(ReportedPositions_DataFrame[column].astype(str).map(len).max(), len(column))
    col_idx = ReportedPositions_DataFrame.columns.get_loc(column)
    writerReportedPositions_DataFrame.sheets['Arkiv'].set_column(col_idx, col_idx, column_length) 


In [228]:
writerReportedPositions_DataFrame.save()